In [95]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [96]:
# Load Dataset 
df = pd.read_csv("customer_support_tickets_with_complaint_date.csv")

# Display basic info and first few rows
df.head()

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating,Complaint Date
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN,2023-06-01 09:01:23
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN,2023-06-01 12:36:22
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0,2023-05-27 10:47:12
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0,2023-05-31 20:24:01
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0,2023-05-25 23:32:14


In [97]:
# Check structure and missing values
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Name                 8469 non-null   object 
 2   Customer Email                8469 non-null   object 
 3   Customer Age                  8469 non-null   int64  
 4   Customer Gender               8469 non-null   object 
 5   Product Purchased             8469 non-null   object 
 6   Date of Purchase              8469 non-null   object 
 7   Ticket Type                   8469 non-null   object 
 8   Ticket Subject                8469 non-null   object 
 9   Ticket Description            8469 non-null   object 
 10  Ticket Status                 8469 non-null   object 
 11  Resolution                    2769 non-null   object 
 12  Ticket Priority               8469 non-null   object 
 13  Tic

Ticket ID                          0
Customer Name                      0
Customer Email                     0
Customer Age                       0
Customer Gender                    0
Product Purchased                  0
Date of Purchase                   0
Ticket Type                        0
Ticket Subject                     0
Ticket Description                 0
Ticket Status                      0
Resolution                      5700
Ticket Priority                    0
Ticket Channel                     0
First Response Time             2819
Time to Resolution              5700
Customer Satisfaction Rating    5700
Complaint Date                  2819
dtype: int64

When the resolution column is null, it indicates that the ticket is still open. In such cases, the time_to_resolution is also null because this metric only applies once the ticket has been resolved by an agent. Similarly, the satisfaction_rate is empty since users can only provide a rating after their issue has been resolved. We'll skip analyzing these columns for now.

Let's move on to the first_response_time column. This field represents the timestamp of the agent’s initial response to the ticket. If the value is null, it means the ticket has not yet been addressed by any agent.


In [98]:
df.columns = df.columns.str.strip()  # Clean column names

In [99]:
# Clean string columns
df['Ticket Status'] = df['Ticket Status'].str.strip().str.lower()
df['Ticket Priority'] = df['Ticket Priority'].str.strip().str.title()
df['Ticket Channel'] = df['Ticket Channel'].str.strip().str.title()

In [100]:
df_resolved = df[df['Ticket Status'] == 'closed'].copy()

In [101]:
# STEP 3: Convert relevant columns to datetime
df['Date of Purchase'] = pd.to_datetime(df['Date of Purchase'], errors='coerce')
df['Complaint Date'] = pd.to_datetime(df['Complaint Date'], errors='coerce')
df['First Response Time'] = pd.to_datetime(df['First Response Time'], errors='coerce')
df['Time to Resolution'] = pd.to_datetime(df['Time to Resolution'], errors='coerce')



In [102]:
# STEP 4: Create time delta metrics (in hours)
df['Response Delay (hrs)'] = (df['First Response Time'] - df['Complaint Date']).dt.total_seconds() / 3600
df['Resolution Delay (hrs)'] = (df['Time to Resolution'] - df['Complaint Date']).dt.total_seconds() / 3600



In [103]:
# Subsets for KPIs
df_response = df.dropna(subset=['Response Delay (hrs)'])
df_resolution = df.dropna(subset=['Resolution Delay (hrs)'])
df_satisfaction = df.dropna(subset=['Customer Satisfaction Rating'])

In [104]:
# KPIs
kpis = {
    "Total Tickets": len(df),
    "Resolved Tickets": df['Ticket Status'].str.lower().eq('closed').sum(),
    "Resolution Rate (%)": round(df['Ticket Status'].str.lower().eq('closed').mean() * 100, 2),
    "Tickets with Response Time": len(df_response),
    "Avg. First Response Time (hrs)": round(df_response['Response Delay (hrs)'].mean(), 2),
    "Tickets with Resolution Time": len(df_resolution),
    "Avg. Time to Resolution (hrs)": round(df_resolution['Resolution Delay (hrs)'].mean(), 2),
    "Tickets with Satisfaction Rating": len(df_satisfaction),
    "Avg. Customer Satisfaction": round(df_satisfaction['Customer Satisfaction Rating'].mean(), 2)
}

# Display KPIs
for k, v in kpis.items():
    print(f"{k}: {v}")

Total Tickets: 8469
Resolved Tickets: 2769
Resolution Rate (%): 32.7
Tickets with Response Time: 5650
Avg. First Response Time (hrs): 82.16
Tickets with Resolution Time: 2769
Avg. Time to Resolution (hrs): 87.44
Tickets with Satisfaction Rating: 2769
Avg. Customer Satisfaction: 2.99


In [105]:
# Feature Engineering Continuation

# 1. Age Grouping
age_bins = [0, 25, 40, 60, 100]
age_labels = ['<25', '25-40', '40-60', '60+']
df['Customer Age Group'] = pd.cut(df['Customer Age'], bins=age_bins, labels=age_labels, right=False)

In [106]:
# 2. Satisfaction Bucketing
satisfaction_bins = [0, 2, 4, 5]
satisfaction_labels = ['Low', 'Medium', 'High']
df['Satisfaction Category'] = pd.cut(df['Customer Satisfaction Rating'], bins=satisfaction_bins, labels=satisfaction_labels, right=True)

In [107]:
# 3. SLA Breach Indicator (Assuming SLA: respond within 24h)
df['SLA Breached'] = df['Response Delay (hrs)'] > 24

# SLA thresholds
SLA_RESPONSE_HOURS = 24
SLA_RESOLUTION_HOURS = 72

# Boolean SLA breach columns
df['SLA Breached - Response'] = df['Response Delay (hrs)'] > SLA_RESPONSE_HOURS
df['SLA Breached - Resolution'] = df['Resolution Delay (hrs)'] > SLA_RESOLUTION_HOURS


In [108]:
# 4. Time of Week / Weekend Analysis
df['Complaint Date'] = pd.to_datetime(df['Complaint Date'], errors='coerce')
df['Ticket Opened Day'] = df['Complaint Date'].dt.day_name()
df['Ticket Opened Month'] = df['Complaint Date'].dt.month_name()
df['Is Weekend'] = df['Complaint Date'].dt.dayofweek >= 5

In [109]:
# 5. Resolution Time Buckets
res_time_bins = [0, 12, 24, 48, 1e6]
res_time_labels = ['<12h', '12-24h', '24-48h', '>48h']
df['Resolution Time Category'] = pd.cut(df['Resolution Delay (hrs)'], bins=res_time_bins, labels=res_time_labels, right=False)

In [110]:
# 6. Response Time Buckets
resp_time_bins = [0, 4, 12, 24, 1e6]
resp_time_labels = ['<4h', '4-12h', '12-24h', '>24h']
df['Response Time Category'] = pd.cut(df['Response Delay (hrs)'], bins=resp_time_bins, labels=resp_time_labels, right=False)

df.columns.tolist()  # show updated columns after feature engineering

['Ticket ID',
 'Customer Name',
 'Customer Email',
 'Customer Age',
 'Customer Gender',
 'Product Purchased',
 'Date of Purchase',
 'Ticket Type',
 'Ticket Subject',
 'Ticket Description',
 'Ticket Status',
 'Resolution',
 'Ticket Priority',
 'Ticket Channel',
 'First Response Time',
 'Time to Resolution',
 'Customer Satisfaction Rating',
 'Complaint Date',
 'Response Delay (hrs)',
 'Resolution Delay (hrs)',
 'Customer Age Group',
 'Satisfaction Category',
 'SLA Breached',
 'SLA Breached - Response',
 'SLA Breached - Resolution',
 'Ticket Opened Day',
 'Ticket Opened Month',
 'Is Weekend',
 'Resolution Time Category',
 'Response Time Category']

In [ ]:
df.to_csv("customer_support_tickets_enriched_clean.csv", index=False)

In [111]:
# df_response_subset = df[
#     df['Response Delay (hrs)'].notnull() &
#     (df['Response Delay (hrs)'] > 0) &
#     (df['Response Delay (hrs)'] < 720)  # Keep only delays under 30 days
# ].copy()

In [112]:
# df_resolution_subset = df[
#     df['Resolution Delay (hrs)'].notnull() &
#     (df['Resolution Delay (hrs)'] > 0) &
#     (df['Resolution Delay (hrs)'] < 1440)  # Keep only under 60 days
# ].copy()

In [113]:
# df_satisfaction_subset = df[
#     df['Customer Satisfaction Rating'].notnull()
# ].copy()

In [114]:
# df_sla_subset = df[
#     df['Resolution Delay (hrs)'].notnull() &
#     (df['Resolution Delay (hrs)'] > 0)
# ].copy()

In [116]:
# df_response_subset.to_csv("response_time_analysis.csv", index=False)
# df_resolution_subset.to_csv("resolution_time_analysis.csv", index=False)
# df_satisfaction_subset.to_csv("satisfaction_analysis.csv", index=False)
# df_sla_subset.to_csv("sla_analysis.csv", index=False)